# Data Preperation

In [ ]:
import pandas as pd
import numpy as np
data_02 = pd.read_excel('Mighty_ABC_DATA 2020-2021.xlsx', '庫存計畫表')

In [ ]:
data_VD005 = data_02[data_02['VendorCode'] == 'VP021']
data_VD005.head()

In [ ]:
data_VD005 = data_VD005.drop(['VendorCode'], axis=1, inplace=False)
data_VD005.info()

In [ ]:
data_VD005 = data_VD005.iloc[:-1].fillna(0)

In [ ]:
data_VD005.head(5)
col = data_VD005['PartNumber']

### Transposing Data

In [ ]:
data_VD005 = np.transpose(data_VD005[['2020/02/01', '2020/03/01', '2020/04/01',
                         '2020/05/01', '2020/06/01', '2020/07/01',
                         '2020/08/01', '2020/09/01', '2020/10/01',
                         '2020/11/01', '2020/12/01', '2021/01/01',
                         '2021/02/01', '2021/03/01', '2021/04/01',
                         '2021/05/01', '2021/06/01', '2021/07/01']])

In [ ]:
data_VD005.columns = col
index = data_VD005.index
data_VD005.head(5)

### Data Normalizaiton

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

In [ ]:
data_VD005 = preprocessing.normalize(data_VD005)
print("Normalized Data = ", data_VD005)

### Data Standardizing

In [ ]:
from sklearn import preprocessing
x_scale = preprocessing.scale(data_VD005)
x_scale

In [ ]:
data_VD005 = pd.DataFrame(x_scale)
data_VD005.head()

In [ ]:
data_VD005.columns = col

In [ ]:
data_VD005.index = index
data_VD005

In [ ]:
data_VD005.insert(0, 'Date', data_VD005.index)
data_VD005.reset_index(inplace=True, drop=True)
data_VD005.head(5)

# Training Model

In [ ]:
# 驗證函數
from numpy import array
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
    # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [ ]:
from pandas import DataFrame
from pandas import concat
 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

### Shift the data

In [ ]:
# 前面天數會有NAN
ss_VD005 = data_VD005[['1-64-13325-00671H']][:100]
ss_VD005['s-5'] = ss_VD005['1-64-13325-00671H'].shift(5)
ss_VD005['s-4'] = ss_VD005['1-64-13325-00671H'].shift(4)
ss_VD005['s-3'] = ss_VD005['1-64-13325-00671H'].shift(3)
ss_VD005['s-2'] = ss_VD005['1-64-13325-00671H'].shift(2)
ss_VD005['s-1'] = ss_VD005['1-64-13325-00671H'].shift(1)
ss_VD005['s-0'] = ss_VD005['1-64-13325-00671H']
ss_VD005.drop(columns='1-64-13325-00671H', inplace=True)
ss_VD005

In [ ]:
# 驗證數據
# 前面幾次的數據 n_in 去預測後面幾次的數據 n_out
predict_months = 2
values = data_VD005['1-64-13325-00671H'].tolist()
data2 = series_to_supervised(values, n_in=predict_months, n_out=1, dropnan=True) #*************優化過去幾天 預測下一天

In [ ]:
data2

### Split the data into training and testing dataset

In [ ]:
# 取出 X , y
X = data2.iloc[:, [0,1,2]].values
y = data2.iloc[:, [-1]].values
# 資料切割 

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

### Model Building

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

from sklearn.preprocessing import MinMaxScaler, StandardScaler

import warnings
warnings.filterwarnings('ignore')

from scipy import stats
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional

In [ ]:
# 使用 BILSTM 看前看後記憶
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
n_steps = 3
n_features = 1
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu', input_shape=(n_steps, n_features))))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [ ]:
# fit model
history = model.fit(X, y, epochs=500, verbose=1, validation_split=0.2)

# Model Performance

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(16,3))
plt.grid()
plt.plot( history.history.get('loss') , color='red')
plt.show()

In [ ]:
# 使用 Deep Learning 進行預測
# 重要數據
pre_Close = model.predict(X).ravel()
Close = y

np.sum((pre_Close - Close)**2) / len(Close)

In [ ]:
def plot_loss (history, model_name):
    plt.figure(figsize = (10, 6))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Train vs Validation Loss for ' + model_name)
    plt.ylabel('Loss')
    plt.xlabel('epoch')
    plt.legend(['Train loss', 'Validation loss'], loc='upper right')

plot_loss (history, 'Bidirectional LSTM')

# Model Testing

In [ ]:
# Plot test data vs prediction
def plot_future(prediction, model_name, y_test):
    
    plt.figure(figsize=(10, 6))
    
    range_future = len(prediction)

    plt.plot(np.arange(range_future), np.array(y_test), label='Test data')
    plt.plot(np.arange(range_future), np.array(prediction),label='Prediction')

    plt.title('Test data vs prediction for ' + model_name)
    plt.legend(loc='upper left')
    plt.xlabel('Time (month)')
    plt.ylabel('Safety stock consumption ($m^3$/capita.month)')

In [ ]:
predictions = model.predict(X)
plot_future(predictions, 'Bidirectional LSTM', y)